In [1]:
import hashlib
import numpy as np
from numpy import array

completed_lines_hash = set()

completed_lines_hash = set()

#Save processed data to smilespad.txt
new = open("smilespad1.txt", "w")

#Read in data file line by line
for line in open("data.txt", "r"):
  
    #Ensure all smiles in original data file are unique
    hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()
  
    if hashValue not in completed_lines_hash:
        completed_lines_hash.add(hashValue)
        
        #Ensure all SMILES are between 35 and 75 characters in length
        if 34 < len(line) < 75:
            #Add start token "{"
            line = line.rjust(len(line)+1, "{")
            
            #Replace \n character with "E"
            line = line.replace("\n", "E")
            
            #Add padding character "}" at end
            line = line.ljust(76, "}")
            
            line = line + "\n"
            
            #Copy over SMILES satisfying requirements
            new.write(line)
    
#Close files
new.close()

In [2]:
#Read in processed data file
data = open("smilespad.txt", "r").read()

#Create a list of the unique characters in the dataset
chars = list(set(data))

#Get size (in characters) of dataset
data_size = len(data) 

#Get number of unique characters in dataset
vocab_size = len(chars)

#Print dataset properties
print("Vocab size: " + str(vocab_size))
print("Data size: " + str(data_size))
print("Characters in data: " + str(chars))

Vocab size: 54
Data size: 36593696
Characters in data: ['c', 'a', ')', 'p', '=', '4', '[', ']', '+', '#', 't', '1', 'M', '{', 's', 'b', '\\', '.', '7', 'B', 'C', 'Z', '8', '/', '2', 'K', 'g', 'I', 'V', 'e', 'T', 'A', 'S', 'H', 'L', 'i', '@', 'E', '9', 'o', '}', 'r', '(', 'F', '-', 'N', 'n', 'l', '5', 'P', 'R', '3', 'O', '6']


In [3]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Create array from characters in the dataset
values = array(chars)
print("Array of unique characters:")
print(values)

#Create unique, numerical labels for each character between 0 and n-1, where n is the number of unique characters
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print("Array of labels for each character:")
print(integer_encoded)

#Encode characters into a one-hot encoding, resulting in an array of size [num unique chars, num unique chars]
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print("Array of one-hot encoded characters:")
print(onehot_encoded)
print("Size of array of one-hot encoded characters: " + str(onehot_encoded.shape))

Array of unique characters:
['c' 'a' ')' 'p' '=' '4' '[' ']' '+' '#' 't' '1' 'M' '{' 's' 'b' '\\' '.'
 '7' 'B' 'C' 'Z' '8' '/' '2' 'K' 'g' 'I' 'V' 'e' 'T' 'A' 'S' 'H' 'L' 'i'
 '@' 'E' '9' 'o' '}' 'r' '(' 'F' '-' 'N' 'n' 'l' '5' 'P' 'R' '3' 'O' '6']
Array of labels for each character:
[41 39  2 48 16 10 36 38  3  0 51  7 27 52 50 40 37  5 13 19 20 35 14  6
  8 25 43 24 34 42 33 18 32 23 26 44 17 21 15 47 53 49  1 22  4 28 46 45
 11 30 31  9 29 12]
Array of one-hot encoded characters:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Size of array of one-hot encoded characters: (54, 54)


C:\Users\jake\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
#Read in processed data file
data = open("smilespad.txt", "r").read()
#Create a list of the dataset
datalist = list(data)
#Create an array of the dataset
dataarray = array(datalist)
#Fit one-hot encoding to dataarray
dataarray = dataarray.reshape(len(dataarray), 1)
OHESMILES = onehot_encoder.fit_transform(dataarray).astype(int)
print("Size of one-hot encoded array of data: " + str(OHESMILES.shape))
print("One-hot encoded array of data:")
print(OHESMILES)

Size of one-hot encoded array of data: (36593696, 54)
One-hot encoded array of data:
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [5]:
#Save OHESMILES as a (compressed) file
np.savez_compressed("ohesmilespad.npz", OHESMILES)

In [6]:
#Create integer SMILES data
INTSMILES = [np.where(r==1)[0][0] for r in OHESMILES]

In [7]:
#Save INTSMILES as a (compressed) file
np.savez_compressed("intsmilespad.npz", INTSMILES)

In [8]:
#Save array with SMILES character, integer encoding, and one hot encoding (vocabulary)
values = np.reshape(values, (np.shape(values)[0], 1))
vocab = np.concatenate((values, integer_encoded.astype(object)), axis = 1)
vocab = vocab[vocab[:,1].argsort()]
vocabvalues = np.reshape(vocab[:,1], (-1,1))
vocabohe = onehot_encoder.fit_transform(vocabvalues)
vocabencodings = np.concatenate((vocab, vocabohe.astype(object)), axis = 1)
print(np.shape(vocabencodings))

np.save("vocabpad.npy", vocabencodings)

(54, 56)


In [9]:
print(vocabencodings)

[['#' 0 1.0 ... 0.0 0.0 0.0]
 ['(' 1 0.0 ... 0.0 0.0 0.0]
 [')' 2 0.0 ... 0.0 0.0 0.0]
 ...
 ['t' 51 0.0 ... 1.0 0.0 0.0]
 ['{' 52 0.0 ... 0.0 1.0 0.0]
 ['}' 53 0.0 ... 0.0 0.0 1.0]]
